---

# 🥉 LEVEL 1: Access Control Designer

**Difficulty:** Intermediate

## 📚 What You'll Learn

- Unity Catalog permission model
- GRANT and REVOKE syntax
- Difference between SELECT, MODIFY, CREATE
- Schema-level vs table-level permissions

## 🎯 Your Mission

Design the basic access control model for the three teams. No fancy features yet - just get the permissions right.

## 📝 Task Instructions

### Step 1: Understand the Current Tables

First, explore what tables exist:

In [ ]:
-- Check what tables we have from the pipeline
-- Note: Pipeline tables are in 'default' schema, ML tables in 'e2eai_iot_turbine'
%sql
SHOW TABLES IN default;

### Step 2: Design Schema Architecture

**Question:** Should all teams access the same schema, or create separate schemas?

**Current Tables (from Pipeline in `workspace.default`):**
- `turbine` - Turbine metadata
- `sensor_bronze` - Raw sensor data
- `historical_turbine_status` - Historical status records
- `parts` - Parts/maintenance data
- `sensor_hourly` - Hourly aggregated sensor data (Materialized View)
- `turbine_training_dataset` - ML training features (Materialized View)
- `turbine_current_features` - Current feature data (Materialized View)

**Option A - Single Schema (Current State):**
```
default
  ├─ sensor_bronze (raw data)
  ├─ sensor_hourly (aggregated - all teams READ)
  ├─ turbine (metadata - all teams READ)
  ├─ parts (maintenance info - field techs only?)
  └─ turbine_training_dataset (ML features - data scientists only?)
```

**Option B - Multiple Schemas (Recommended for Production):**
```
default (existing pipeline tables)
  └─ [Keep raw/bronze tables here]
  
turbine_operations (Team A - Analysts Group)
  ├─ sensor_hourly_view (aggregated data only)
  ├─ turbine_view (filtered columns)
  └─ parts_view (maintenance schedule)
  
turbine_analytics (Team B - Dataengineers Group)
  ├─ sensor_bronze (full access for ML)
  ├─ turbine_training_dataset (ML features)
  └─ turbine_current_features (real-time features)
  
turbine_executive (Team C - Users Group)
  ├─ kpi_dashboard_view (aggregated KPIs)
  └─ fleet_summary_view (high-level metrics)
```

**Your Task:** Choose your architecture and document it below.

**Implementation Options:**
- **Option A (Simpler):** Use existing `default` and `e2eai_iot_turbine` schemas with table-level permissions
- **Option B (Advanced):** Create new schemas for better separation (requires `CREATE SCHEMA` privilege)

**Note:** Both approaches are valid! Option A is faster to implement, Option B provides better organization for production environments.

### 📝 Your Schema Design

Document your decision:

```
My Architecture Choice: [Option A / Option B / Custom]

Reasoning:
- [Why did you choose this structure?]
- [How does it meet security requirements?]
- [What are the tradeoffs?]

Schema Structure:
Schema 1: _______________
  Purpose: _______________
  Teams with access: _______________
  
Schema 2: _______________
  Purpose: _______________
  Teams with access: _______________

[Add more schemas if needed]
```

In [ ]:
-- Option B: Create new schemas (optional)
CREATE SCHEMA IF NOT EXISTS turbine_operations;
CREATE SCHEMA IF NOT EXISTS turbine_analytics;
CREATE SCHEMA IF NOT EXISTS turbine_executive;

-- Grant schema-level permissions
-- analysts group: READ access to operations schema
GRANT USAGE, SELECT ON SCHEMA turbine_operations TO `analysts`;

-- dataengineers group: FULL access to analytics schema
GRANT USAGE, SELECT, MODIFY, CREATE ON SCHEMA turbine_analytics TO `dataengineers`;

-- users group: READ access to executive schema
GRANT USAGE, SELECT ON SCHEMA turbine_executive TO `users`;

### Step 3: Create User Groups

In Unity Catalog, you manage permissions via **groups**, not individual users.

**Task:** Create groups for each team (or use existing ones):

In [ ]:
-- ✅ Use Account-level groups for GRANT statements
-- Available groups (Source: Account): analysts, dataengineers, admins, users
-- 
-- Only Account-level groups can be used as principals in GRANT statements.
-- Workspace-level groups will NOT work.

SHOW GROUPS;
%sql

### Step 4: Grant Permissions - Team A (Analysts Group)

**Requirements for `analysts` group:**
- READ access to operational data (sensor_hourly, turbine, parts, historical_turbine_status)
- NO access to raw bronze tables
- NO WRITE access (read-only)

**Implementation:**


In [ ]:
-- GRANT statements for analysts group
-- Note: Pipeline tables are in 'default' schema

-- Option A: Table-level permissions (more granular control)
%sql
GRANT SELECT ON TABLE default.sensor_hourly TO `analysts`;
GRANT SELECT ON TABLE default.turbine TO `analysts`;
GRANT SELECT ON TABLE default.parts TO `analysts`;
GRANT SELECT ON TABLE default.historical_turbine_status TO `analysts`;

-- Option B (if you created turbine_operations schema):

-- GRANT USAGE, SELECT ON SCHEMA turbine_operations TO `analysts`;SHOW GRANTS ON TABLE default.sensor_hourly;

-- Verify grants

### Step 5: Grant Permissions - Team B (Dataengineers Group)

**Requirements for `dataengineers` group:**
- READ access to ALL sensor data (including raw bronze tables)
- WRITE access to analytics/ML tables
- SELECT + MODIFY on schemas for experimentation

**Implementation:**

In [ ]:
-- GRANT statements for dataengineers group
-- Note: Pipeline tables in 'default', ML tables in 'e2eai_iot_turbine'

%sql
-- Option A: Schema-level permissions (grants access to all tables in schema)
GRANT SELECT ON SCHEMA default TO `dataengineers`;
GRANT SELECT, MODIFY ON SCHEMA e2eai_iot_turbine TO `dataengineers`;

-- Option B (if you created turbine_analytics schema):
-- GRANT USAGE, SELECT, MODIFY, CREATE ON SCHEMA turbine_analytics TO `dataengineers`;

-- Verify grants
SHOW GRANTS ON SCHEMA default;

### Step 6: Grant Permissions - Team C (Users Group)

**Requirements for `users` group:**
- READ access ONLY to aggregated views/dashboards
- NO access to raw sensor data
- NO access to technical tables

**Strategy:** Create views first, then grant access to views only.

**Alternative:** Grant to your personal account instead of the users group.

In [ ]:
-- First, create an executive KPI view
-- Note: sensor_hourly is in 'default' schema from pipeline
CREATE OR REPLACE VIEW default.executive_kpi_dashboard AS
SELECT
  DATE(window.start) as date,
  COUNT(DISTINCT turbine_id) as active_turbines,
  SUM(avg_power) as total_power_output_kw,
  AVG(avg_power) as avg_power_per_turbine
  -- Add more KPIs but NO raw sensor data!
FROM default.sensor_hourly
GROUP BY DATE(window.start);

In [ ]:
-- GRANT access to view only (NOT underlying tables)
-- Option 1: Use 'users' group (Account-level group)
%sql
GRANT SELECT ON VIEW default.executive_kpi_dashboard TO `users`;

-- Option 2: Use 'analysts' group instead
-- GRANT SELECT ON VIEW default.executive_kpi_dashboard TO `analysts`;

-- GRANT USAGE, SELECT ON SCHEMA turbine_executive TO `users`;

-- Option 3 (if you created turbine_executive schema):-- Create view in executive schema, then grant schema access

### Step 7: Verify Permissions

Check what permissions you've granted:

In [ ]:
-- Check grants for a specific table
%sql
SHOW GRANTS ON TABLE default.sensor_hourly;

In [ ]:
-- Check grants for a schema
%sql
SHOW GRANTS ON SCHEMA default;

## 📊 Success Criteria

✅ Created user groups for all 3 teams  
✅ Granted appropriate READ permissions  
✅ Restricted WRITE access to data scientists only  
✅ Created executive view with aggregated data only  
✅ Verified permissions with SHOW GRANTS  

**Level 1 Complete!** 🎉 You've designed basic access control!

---

# 🥈 LEVEL 2: Security Architect

**Difficulty:** Advanced

## 📚 What You'll Learn

- Data masking techniques
- Row-level security (filtering)
- Column-level access control
- Handling edge cases (users in multiple groups)

## 🎯 Your Mission

The basic permissions work, but there are problems:
1. Field techs can see turbine purchase costs (confidential!)
2. Execs accidentally saw raw sensor errors (confusing!)
3. A user in multiple groups has wrong permissions

**Goal:** Implement advanced security features to handle these issues.

## 📝 Task Instructions

### Step 1: Identify Sensitive Columns

**Review your tables and identify sensitive data:**

```
Table: parts
- Potentially sensitive: supplier info, cost data
- Who should see: Field techs (operational), executives (costs)

Table: sensor_bronze
- Contains: Raw sensor data with possible errors/noise
- Who should see: Data scientists only (for ML)
- Field techs: Should use sensor_hourly (cleaned)

Table: sensor_hourly
- Contains: Aggregated, cleaned sensor data
- Who should see: All teams (operational data)


Table: turbine_training_dataset**Your Task:** List sensitive columns in your tables.

- Contains: ML features, may include sensitive patterns

- Who should see: Data scientists only```

- Who should see: All teams (different views)

Table: turbine, historical_turbine_status- Contains: Turbine metadata and status

### 📝 Your Sensitive Data Audit

Document what needs protection:

```
Table: _______________________
Column: _______________________
  Sensitivity: [Confidential / Internal / Public]
  Accessible by: [Which teams?]
  Masking strategy: [Redact / Hash / Aggregate / Filter]

[Repeat for all sensitive columns]
```

### Step 2: Implement Data Masking

Unity Catalog supports **column masking** using SQL functions.

**Strategy:** Create masked views for teams that shouldn't see raw data.

In [ ]:
-- Create a masked view for Field Technicians
-- Hide cost information from them

CREATE OR REPLACE VIEW default.sensor_hourly_field_view AS
SELECT
  turbine_id,
  window,
  avg_power,
  avg_vibration,
  avg_temperature,
  -- Mask any cost-related fields
  '***REDACTED***' as cost_per_hour  -- If this column exists
FROM default.sensor_hourly;

In [ ]:
-- Now grant analysts access to MASKED VIEW instead of raw table
%sql
REVOKE SELECT ON TABLE default.sensor_hourly FROM `analysts`;
GRANT SELECT ON VIEW default.sensor_hourly_field_view TO `analysts`;

### Step 3: Implement Row-Level Security

**Scenario:** Different field technicians work in different regions. They should only see turbines in THEIR region.

**Implementation:** Create views with row filters based on user attributes.

In [ ]:
-- First, create a user-region mapping table
CREATE OR REPLACE TABLE default.user_regions (
  username STRING,
  allowed_region STRING
);

-- Insert sample data
INSERT INTO default.user_regions VALUES
  ('tech_chicago@company.com', 'Chicago'),
  ('tech_miami@company.com', 'Miami'),
  ('tech_nyc@company.com', 'New York');

In [ ]:
-- Create row-filtered view based on current user
CREATE OR REPLACE VIEW default.sensor_hourly_my_region AS
SELECT s.*
FROM default.sensor_hourly s
-- Join with turbine location info (assuming location column exists)
-- INNER JOIN default.turbine t ON s.turbine_id = t.turbine_id
WHERE 
  -- Filter based on current user's region
  -- t.location IN (
  --   SELECT allowed_region 
  --   FROM default.user_regions 
  --   WHERE username = current_user()
  -- )
  
  -- Simplified version if location is in sensor_hourly:
  1=1  -- Replace with actual row filter logic
;

### Step 4: Column-Level Security

**Advanced:** Different users see different columns in the same table.

**Example:** Data scientists see all columns, field techs see subset.

In [ ]:
-- Create role-specific views with different column sets

-- View for Field Technicians (limited columns from turbine)
CREATE OR REPLACE VIEW default.turbine_field_view AS
SELECT
  turbine_id,
  model,
  location,
  lat,
  long
  -- Exclude any internal/sensitive fields if they exist
FROM default.turbine;

-- View for Data Scientists (all columns including ML features)
CREATE OR REPLACE VIEW default.turbine_analytics_view AS
SELECT
  t.*,
  tf.avg_energy,

  tf.std_energy,LEFT JOIN default.turbine_current_features tf ON t.turbine_id = tf.turbine_id;

  tf.trend_energyFROM default.turbine t

In [ ]:
-- Grant appropriate views to each group (using built-in groups)
%sql
GRANT SELECT ON VIEW default.turbine_field_view TO `analysts`;
GRANT SELECT ON VIEW default.turbine_analytics_view TO `dataengineers`;

### Step 5: Handle Edge Cases

**🚨 Problem:** Sarah is BOTH a Field Technician AND a Data Scientist.

**Question:** What permissions does she get?
- Most permissive (Data Scientist level)? ✓ Unity Catalog default
- Most restrictive (Field Technician level)? ✗ Not default
- Custom combination? (Advanced)

**Your Task:** Document how you would handle this.

### 📝 Your Edge Case Strategy

**Scenario Analysis:**

**Case 1: User in Multiple Groups**
```
User: sarah@company.com
Groups: analysts, dataengineers
Expected behavior: Most permissive (Unity Catalog default)
  - Gets SELECT from analysts group
  - Gets SELECT + MODIFY from dataengineers group
  - Final permissions: SELECT + MODIFY (most permissive wins)
Your implementation: Unity Catalog automatically handles this
```

**Case 2: Temporary Access**
```
A contractor needs 1-week access to specific turbine data.
Your solution: 
  - Create temporary user account
  - Add to analysts group
  - Set calendar reminder to remove after 1 week
  - Or grant directly to their email, revoke after period
```

**Case 3: Emergency Override**
```
During an outage, operations team needs immediate access to ALL data.
Your solution: 
  - Add user to admins group temporarily
  - Document in permission_tracking table
  - Revoke after incident resolved
  - Review access in post-mortem
```

## 📊 Success Criteria

✅ Implemented data masking for sensitive columns  
✅ Created row-level security filters  
✅ Set up column-level access control  
✅ Documented edge case handling strategy  
✅ Validated security with test queries  

**Level 2 Complete!** 🎉 You're now a security architect!

---

# 🥇 LEVEL 3: Real-World Application & Best Practices

**Difficulty:** Advanced | **Time:** 45 minutes

> 💡 **Free Edition Note:** This level focuses on **documentation and manual processes** since automated enterprise features (system audit logs, alerting) require Unity Catalog Pro/Enterprise.

## 📚 What You'll Learn

- Document governance decisions for your team
- Create manual permission backup/restore procedures
- Understand data lineage and impact analysis
- Design access review workflows
- Apply governance principles to real scenarios

## 🎯 Your Mission

You've implemented technical controls in Level 1 & 2. Now make your governance framework **maintainable, auditable, and team-ready**.

1. Automated compliance reporting
2. Data lineage tracking
3. Regular access reviews
4. Incident response procedures

**Goal:** Build a production-ready governance system that scales.

## 📝 Implementation Guide

### Component 1: Governance Documentation

**Create a governance playbook that includes:**


### 📋 Your Governance Playbook

Complete this template:

---

## **WIND TURBINE DATA GOVERNANCE PLAYBOOK**

### 1. Access Control Matrix

| Group Name | Source | Schema Access | Table Access | Permission Level | Approval Required |
|------------|--------|---------------|--------------|------------------|-------------------|
| `analysts` | Account ✅ | default | sensor_hourly, turbine, parts | SELECT | Manager approval |
| `dataengineers` | Account ✅ | default, e2eai_iot_turbine | ALL | SELECT, MODIFY | Lead approval |
| `users` | Account ✅ | default | KPI views only | SELECT | Management approval |
| `admins` | Account ✅ | ALL | ALL | ALL | Security team |

⚠️ **Note:** Only Account-level groups work with GRANT statements. Workspace-level groups cannot be used as principals.

### 2. Data Classification

| Classification | Examples | Who Can Access | Encryption | Audit Level |
|----------------|----------|----------------|------------|-------------|
| Public | Fleet size, location names | All employees | No | Basic |
| Internal | Sensor readings | Tech + DS + Execs | TLS | Standard |
| Confidential | Cost data, failure predictions | Execs + DS only | TLS + at-rest | Enhanced |
| Restricted | Strategic plans | Execs only | Full encryption | Complete |

### 3. Incident Response Procedures

**Scenario: Unauthorized Access Detected**
1. Step 1: Immediately revoke access using `REVOKE` statement
2. Step 2: Check audit logs with `SHOW GRANTS` queries
3. Step 3: Notify security team
4. Step 4: Review similar permissions for other users

**Scenario: Data Leak**
1. Step 1: Identify affected tables/schemas
2. Step 2: Revoke all external access immediately
3. Step 3: Review data lineage to find downstream impacts
4. Step 4: Restore from grants_backup if needed

### 4. Access Review Schedule

- **Monthly:** Review new user additions
- **Quarterly:** Audit all analyst/dataengineer group membership
- **Annually:** Complete access recertification
- **Ad-hoc:** After employee role changes

### 5. Compliance Checklist

- [ ] GDPR: Personal data minimized
- [ ] SOC 2: Access controls documented
- [ ] ISO 27001: Periodic access reviews
- [ ] Internal: Cost data protected
- [ ] Internal: Audit trail enabled

---

### Component 2: Manual Permission Tracking

**Task:** Create a manual tracking system for permissions (since system tables aren't available in Free Edition).

In [ ]:
-- Create a manual permission tracking table in default schema

CREATE TABLE IF NOT EXISTS default.permission_tracking (
  grant_date TIMESTAMP,
  grantee STRING,
  object_type STRING,  -- 'SCHEMA', 'TABLE', 'VIEW'
  object_name STRING,
  privilege STRING,  -- 'SELECT', 'MODIFY', 'ALL'
  granted_by STRING,
  justification STRING,
  review_date TIMESTAMP
);

-- Example: Log a permission grant manually
%sql
INSERT INTO default.permission_tracking VALUES (
  current_timestamp(),
  'analysts',
  'TABLE',
  'default.sensor_hourly',
  'SELECT',
  current_user(),
  'Operational data access for analysts group',
  current_timestamp() + INTERVAL 90 DAYS
);

In [ ]:
-- Query your permission tracking table

SELECT 
  grantee,
  object_name,
  privilege,
  grant_date,
  review_date,
  DATEDIFF(review_date, current_timestamp()) as days_until_review
FROM default.permission_tracking
WHERE review_date < current_timestamp() + INTERVAL 30 DAYS
ORDER BY review_date;

-- This helps you identify permissions that need review soon

### Component 3: Data Lineage Documentation

**Purpose:** Understand data flow and impact of changes.

**Task:** Manually document your data lineage (Unity Catalog Free Edition has basic lineage in UI).

### 📊 Your Data Lineage Map

Document your actual data flows:

```
SOURCE → TRANSFORMATION → DESTINATION → CONSUMERS
=================================================

Flow 1: Sensor Data Pipeline
sensor_bronze (raw IoT stream)
  → sensor_hourly (hourly aggregation via MATERIALIZED VIEW)
  → executive_kpi_dashboard (aggregated view)
  → Used by: Executives team
  
Impact Analysis:
- If sensor_bronze schema changes → sensor_hourly breaks
- If sensor_hourly changes → executive dashboard breaks
- Affected teams: Field Techs, Data Scientists, Executives

Flow 2: ML Training Pipeline
sensor_bronze → turbine_training_dataset → ML Model Training
  → Used by: Data Scientists team
  
Impact Analysis:
- Schema changes affect model training
- Need to retrain models if features change

Flow 3: Operational Data

turbine (metadata) + parts (maintenance) → Field Tech dashboards- Update after each schema/pipeline change

  → Used by: Field Technicians team- Document dependencies in this notebook or a shared document

- Check Unity Catalog UI → Data Explorer → Select table → "Lineage" tab

[YOUR TASK: Document your actual lineage here]**Manual Lineage Tracking:**

```

### Component 4: Manual Access Reviews

**Task:** Create queries to support quarterly access reviews (using SHOW GRANTS since system tables aren't available).

In [ ]:
-- Query 1: Check all grants on sensitive tables manually

-- Check sensor_bronze (should be data scientists only)
SHOW GRANTS ON TABLE default.sensor_bronze;

-- Check parts table
SHOW GRANTS ON TABLE default.parts;

-- Check training dataset
SHOW GRANTS ON TABLE default.turbine_training_dataset;

-- YOUR TASK: Document results in permission_tracking table

In [ ]:
-- Query 2: Review all schema-level grants

SHOW GRANTS ON SCHEMA default;

-- Review table-level grants for key tables
SHOW GRANTS ON TABLE default.sensor_hourly;
SHOW GRANTS ON TABLE default.turbine;

-- YOUR TASK: Run these quarterly and document in permission_tracking

### 📋 Access Review Checklist

Create a quarterly review process:

**Every Quarter:**
1. Run `SHOW GRANTS` on all sensitive tables
2. Review permission_tracking table for upcoming review dates
3. Verify each group still needs their current access
4. Document review in permission_tracking:
   ```sql
   UPDATE default.permission_tracking
   SET review_date = current_timestamp() + INTERVAL 90 DAYS
   WHERE grantee = 'team_name' AND object_name = 'table_name';
   ```
5. Revoke unnecessary permissions
6. Share review results with management

**Red Flags to Check:**
- Users with access to tables they don't use
- Broad schema-level grants when table-level would suffice
- Groups with MODIFY rights that only need SELECT

### Component 5: Permission Backup & Recovery

**Scenario:** You accidentally revoke critical permissions! How do you recover?

**Task:** Create a manual backup procedure.

In [ ]:
-- Manual backup strategy (Free Edition compatible)

-- Step 1: Export all current grants to a table
CREATE TABLE IF NOT EXISTS default.grants_backup (
  backup_date TIMESTAMP,
  object_type STRING,
  object_name STRING,
  grant_statement STRING
);

-- Step 2: Manually record key grants (using built-in groups)
%sql
INSERT INTO default.grants_backup VALUES
  (current_timestamp(), 'TABLE', 'default.sensor_hourly', 'GRANT SELECT ON TABLE default.sensor_hourly TO `analysts`;'),
  (current_timestamp(), 'TABLE', 'default.turbine', 'GRANT SELECT ON TABLE default.turbine TO `analysts`;'),
  (current_timestamp(), 'SCHEMA', 'default', 'GRANT SELECT ON SCHEMA default TO `dataengineers`;'),
  (current_timestamp(), 'SCHEMA', 'e2eai_iot_turbine', 'GRANT SELECT, MODIFY ON SCHEMA e2eai_iot_turbine TO `dataengineers`;');

-- YOUR TASK: Add all your GRANT statements here

In [ ]:
-- Restore procedure: Query and execute

SELECT grant_statement
FROM default.grants_backup
WHERE backup_date = (SELECT MAX(backup_date) FROM default.grants_backup)
ORDER BY object_name;

-- Copy the output and execute each GRANT statement manually
-- This restores your permissions to the last backup state

### Component 6: Manual Compliance Reporting

**Task:** Create a manual compliance checklist (since automated reporting needs system tables).

### 📊 Quarterly Compliance Report Template

**Report Date:** _______
**Reviewed By:** _______

#### 1. Access Control Coverage
- [ ] All sensitive tables have explicit grants (sensor_bronze, parts, turbine_training_dataset)
- [ ] No overly broad schema-level grants
- [ ] Analysts: SELECT only on operational tables
- [ ] Dataengineers: SELECT on all data, SELECT + MODIFY on e2eai_iot_turbine schema
- [ ] Users/Executives: SELECT only on aggregated views

#### 2. Permission Inventory
Run these queries and document results:
```sql
SHOW GRANTS ON SCHEMA default;
SHOW GRANTS ON SCHEMA e2eai_iot_turbine;
SHOW GRANTS ON TABLE default.sensor_bronze;
SHOW GRANTS ON TABLE default.sensor_hourly;
SHOW GRANTS ON TABLE default.turbine;
```

**Total Groups with Access:** _____
**Total Tables Protected:** _____

#### 3. Data Masking Verification
- [ ] Executive views show aggregated data only (no raw sensor values)
- [ ] Analyst views exclude sensitive columns
- [ ] Test queries validated

#### 4. Violations Found
List any issues:
1. _______
2. _______

#### 5. Remediation Actions
1. _______
2. _______

### Component 7: Incident Response Playbook

**Task:** Set up alerts for security violations.

**Pseudo-code for alert system:**

```python
# Alert Rule 1: Unusual access patterns
if user_access_count > 1000 in last_hour:
    send_alert(f"User {username} accessed {table} {count} times")

# Alert Rule 2: Sensitive table access by unauthorized user
if accessed_table == 'sensor_bronze' and user_group not in ['dataengineers', 'admins']:
    send_critical_alert(f"Unauthorized access attempt by {username}")

# Alert Rule 3: Permission changes
if action in ['GRANT', 'REVOKE']:
    send_notification(f"Permission change: {action} {table} to {user}")
```

In Databricks, implement with:
- **Databricks SQL Alerts** on audit log queries
- **Webhooks** to Slack/Email/PagerDuty
- **Jobs** that run hourly/daily checks

## ✅ Validation Steps

### Governance Framework Checklist

1. **Documentation Complete**
   - [ ] Access control matrix filled out
   - [ ] Data classification defined
   - [ ] Incident response procedures documented
   - [ ] Compliance checklist created

2. **Technical Implementation**
   - [ ] Permission tracking table created
   - [ ] Data lineage documented
   - [ ] Manual access review queries working
   - [ ] Grant backup system in place
   - [ ] Quarterly compliance template ready

3. **Operational Readiness**
   - [ ] Team trained on playbook
   - [ ] Quarterly review schedule set
   - [ ] Incident response procedures documented
   - [ ] Permission backup tested

### Test Your System

```sql
-- Test 1: Check permission tracking
SELECT * FROM default.permission_tracking 
ORDER BY grant_date DESC LIMIT 10;

-- Test 2: Verify grants on sensitive tables
SHOW GRANTS ON TABLE default.sensor_bronze;
SHOW GRANTS ON TABLE default.parts;

-- Test 3: Test restore from backup
SELECT grant_statement FROM default.grants_backup
WHERE backup_date = (SELECT MAX(backup_date) FROM default.grants_backup);
-- Copy and test one GRANT statement

-- Test 4: Check upcoming reviews
SELECT * FROM default.permission_tracking
WHERE review_date < current_timestamp() + INTERVAL 30 DAYS;
```

## 📊 Success Criteria

✅ Complete governance playbook documented  
✅ Manual permission tracking operational  
✅ Data lineage mapped and documented  
✅ Manual access review process created  
✅ Grant backup and restore system working  
✅ Compliance reporting template created  
✅ Incident response procedures defined  

**Level 3 Complete!** 🏆 You've created a maintainable governance framework!

> 💡 **Free Edition Reality Check:** This level used manual processes suitable for Free Edition. In Pro/Enterprise, many of these would be automated via system tables, audit logs, and alerting.

---

# 🎓 Final Reflection & Next Steps

## 🎉 Congratulations!

You've built a **production-ready data governance framework** from scratch! 

### What You've Mastered

**Technical Skills:**
- ✅ Unity Catalog permission model (GRANT/REVOKE)
- ✅ Data masking and column-level security
- ✅ Row-level security with filtered views
- ✅ Audit logging and compliance reporting
- ✅ Data lineage analysis
- ✅ Disaster recovery procedures

**Governance Principles:**
- ✅ Principle of Least Privilege
- ✅ Defense in Depth (multiple security layers)
- ✅ Auditability and Transparency
- ✅ Separation of Duties
- ✅ Regular Access Reviews

---

## 🚀 Real-World Application

### How This Works in Production

**Scenario: New Employee Onboarding**
```
1. HR creates user account
2. Manager requests access via ticket
3. Governance team reviews playbook
4. Execute GRANT statements
5. User added to audit log
6. Confirmation email sent
```

**Scenario: Security Incident**
```
1. Alert triggered: Unusual access pattern
2. Security team reviews audit logs
3. Identify affected tables via lineage
4. REVOKE access immediately
5. Restore from grants backup if needed
6. Root cause analysis
```

**Scenario: Compliance Audit**
```
1. Auditor requests access reports
2. Run compliance dashboard queries
3. Export results to PDF
4. Demonstrate access controls
5. Show audit trail
6. Pass audit ✓
```

---

## 💡 Advanced Topics (For Later)

Once you've mastered the basics, explore:

### 1. **Attribute-Based Access Control (ABAC)**
Instead of role-based, use user attributes:
```sql
-- Grant access based on user location, department, clearance level
CREATE VIEW filtered_view AS
SELECT * FROM sensor_data
WHERE location = get_user_attribute('location')
  AND clearance_level >= get_user_attribute('clearance');
```

### 2. **Dynamic Data Masking**
Mask data based on context:
```sql
-- Show full data in production, masked in dev
CASE 
  WHEN current_database() = 'prod' THEN actual_value
  ELSE mask(actual_value)
END
```

### 3. **Time-Based Access**
Temporary permissions that auto-expire:
```sql
-- Grant access for 24 hours only
-- (Requires external automation)
```

### 4. **Cross-Catalog Governance**
Manage permissions across multiple catalogs:
```sql
GRANT SELECT ON * TO `analysts`;
GRANT SELECT ON dev.* TO `data_engineers`;
```

---

## 🎯 Next Steps in Your Journey

### Immediate Actions
1. **Apply to Your Data:** Use this framework on your actual datasets
2. **Share with Team:** Present your governance model to stakeholders
3. **Get Feedback:** What security concerns do they have?

### Continue Learning
- **Module 03 - BI Dashboards:** Create compliance dashboards
- **Module 04 - ML:** Governance for ML models and features
- **Module 05 - GenAI:** Security for AI agents accessing data

### Certification Prep
This challenge prepares you for:
- **Databricks Data Engineer Associate**
- **Databricks Data Analyst Associate**
- **Security/Compliance certifications**

---

## 📚 Resources

### Documentation
- [Unity Catalog Best Practices](https://docs.databricks.com/data-governance/unity-catalog/best-practices.html)
- [Data Governance Guide](https://docs.databricks.com/data-governance/index.html)
- [Audit Logs](https://docs.databricks.com/administration-guide/account-settings/audit-logs.html)

### Community
- Databricks Community Forums (Data Governance section)
- Unity Catalog GitHub discussions
- LinkedIn #DataGovernance

---

## 🏆 Challenge Badge Unlocked!

**🔐 Data Governance Master**

You've completed:
- ✅ Level 1: Access Control Designer
- ✅ Level 2: Security Architect  
- ✅ Level 3: Governance Master

**Share your achievement:**
> "Just completed a comprehensive data governance challenge covering Unity Catalog, data masking, audit logging, and compliance reporting! #DataGovernance #Databricks #UnityCatalog"

---

**Thank you for your dedication to secure, compliant data engineering!** 🎉

*Remember: Good governance isn't about blocking access - it's about enabling safe, responsible data use.*